<a href="https://colab.research.google.com/github/alekhyabobba/Pyspark-with-python/blob/main/Pyspark_Ml_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pyspark ML 

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 50 kB/s 
     |████████████████████████████████| 199 kB 53.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=30cb4538d836760cdc6cbcd3eed5b4d7b04b56529da25d14d08d9000c8a5aec5
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Mlib').getOrCreate()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df = spark.read.csv("/content/drive/My Drive/mlib_data_practice_pyspark.csv",header = True, inferSchema = True)

In [7]:
df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [9]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [10]:
df.columns

['Name', 'age', 'Experience', 'Salary']

In [13]:
from pyspark.ml.feature import VectorAssembler
featuresassembler = VectorAssembler(inputCols =["age","Experience"],outputCol="independent_features")

In [14]:
output = featuresassembler.transform(df)

In [15]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|independent_features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [16]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'independent_features']

In [17]:
finalized_data = output.select("independent_features","Salary")

In [18]:
finalized_data.show()

+--------------------+------+
|independent_features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [20]:
from pyspark.ml.regression import LinearRegression
#train test split
train_data,test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol = 'independent_features',labelCol ="Salary")
regressor = regressor.fit(train_data)

In [21]:
regressor.coefficients

DenseVector([-90.5483, 1608.7819])

In [22]:
regressor.intercept

16079.13669064716

In [23]:
#Predictions
pred_results = regressor.evaluate(test_data)

In [27]:
pred_results.predictions.show()

+--------------------+------+------------------+
|independent_features|Salary|        prediction|
+--------------------+------+------------------+
|          [23.0,2.0]| 18000|17214.090796328448|
+--------------------+------+------------------+



In [29]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(785.909203671552, 617653.276415653)